In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import tensorflow as tf

import deepxde as dde


C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework

In [2]:
def ode_system(x, y):
        
        dy_x = tf.gradients(y, x)[0]
 
        
        return [dy_x - 2*x]


In [3]:
def boundary(_, on_initial):
    
    return on_initial


In [4]:
 def func(x):
        """
        y = 2x
        y(0)=0
        """
        return x**2
        #return np.hstack((np.sin(x), np.cos(x)))

In [5]:
geom = dde.geometry.Interval(0, 1)

In [6]:
bc1 = dde.DirichletBC(geom, func, boundary, component=0)
bc2 = dde.DirichletBC(geom, func, boundary, component=1)

In [7]:
data = dde.data.PDE(
        geom, ode_system, [bc1,bc2], 35, 2, solution=func, num_test=100
    )

In [8]:
layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

Building feed-forward neural network...
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
'build' took 0.088010 s



In [9]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs=20000)

Compiling model...
'compile' took 0.445823 s

Initializing variables...
Training model...

Step      Train loss                        Test loss                         Test metric   
0         [1.70e+00, 7.01e-01, nan]         [1.72e+00, 0.00e+00, 0.00e+00]    [1.24e+00]    


C:\Users\Hasin Ishrak\Desktop\deepxde\deepxde\model.py:351: RuntimeWarning: invalid value encountered in less
  if self.best_loss_train > np.sum(self.loss_train):


1000      [1.68e-05, 1.75e-09, nan]         [1.36e-05, 0.00e+00, 0.00e+00]    [6.67e-04]    
2000      [7.81e-06, 8.63e-11, nan]         [7.10e-06, 0.00e+00, 0.00e+00]    [4.69e-04]    
3000      [5.93e-06, 1.06e-10, nan]         [5.39e-06, 0.00e+00, 0.00e+00]    [4.11e-04]    
4000      [4.30e-06, 1.11e-10, nan]         [3.89e-06, 0.00e+00, 0.00e+00]    [3.51e-04]    
5000      [3.22e-06, 8.61e-11, nan]         [2.90e-06, 0.00e+00, 0.00e+00]    [3.03e-04]    
6000      [2.48e-06, 5.84e-11, nan]         [2.23e-06, 0.00e+00, 0.00e+00]    [2.67e-04]    
7000      [1.96e-06, 1.68e-10, nan]         [1.77e-06, 0.00e+00, 0.00e+00]    [2.47e-04]    
8000      [3.06e-06, 6.92e-07, nan]         [2.91e-06, 0.00e+00, 0.00e+00]    [1.35e-03]    
9000      [1.35e-06, 7.89e-10, nan]         [1.22e-06, 0.00e+00, 0.00e+00]    [2.24e-04]    
10000     [3.05e-05, 1.34e-05, nan]         [3.06e-05, 0.00e+00, 0.00e+00]    [6.07e-03]    
11000     [1.02e-06, 1.16e-09, nan]         [9.16e-07, 0.00e+00, 0.00e